In [1]:
from pf_simulation import PowerFactorySim as pfsim

FOLDER_NAME = ''
PROJECT_NAME = '39 Bus New England System'
STUDY_CASE_NAME = '1. Power Flow'
MONITORED_VARIABLES = {
    '*.ElmTerm': ['m:u1', 'm:phiu'],
    '*.ElmSym': ['c:firel', 's:outofstep']
}
VAR_FOLDER = {
    '*.ElmTerm': ['magnitude','angle'],
    '*.ElmSym': ['rotor_angle', 'out_of_step']
}

#sim = PowerFactorySim(folder_name=FOLDER_NAME, project_name=PROJECT_NAME, study_case_name=STUDY_CASE_NAME)
sim = pfsim(folder_name=FOLDER_NAME,
            project_name=PROJECT_NAME,
            study_case_name=STUDY_CASE_NAME)
# sim.app.Show()

In [2]:
import os
import pandas as pd

# Membuat skenario operasi
f_line = sim.app.GetCalcRelevantObjects('*.ElmLne')
f_location = [0, 20, 50, 80]
f_duration = [0.1, 0.15, 0.18, 0.2, 0.25]
load_level = [i/100 for i in range(50,115,5)]

# Memuat nilai default load dan gen
file_dir = "output/default_data"
load_file = f"{file_dir}/load_data.parquet"
gen_file = f"{file_dir}/generator_data.parquet"
if all(map(os.path.exists, [load_file, gen_file])):
    load_default, gen_default = (pd.read_parquet(load_file), pd.read_parquet(gen_file))
else:
    load_default, gen_default = sim.loadGenData()
    load_default.to_parquet(load_file, index=False)
    gen_default.to_parquet(gen_file, index=False)

# Mengatur parameter untuk simulasi optimal power flow
sim.opfSetup()

# Mengatur parameter beban ke nilai default
sim.loadSetup(1, load_default)
sim.opfAnalysis('los')
sim.ldfAnalysis()

Testing OPF

In [3]:
file_dir = "output/generator_dispatch"
for load_lvl_iter in load_level:
    sim.loadSetup(load_lvl_iter, load_default)
    try:
        sim.opfAnalysis('los')
    except AttributeError:
        print(f'{load_lvl_iter:.2f} load, OPF min los failed, trying min dev')
        sim.opfAnalysis('dev')
    
    # Lakukan analisis daya dan ambil dispatch
    gen_dispatch = sim.ldfAnalysis().gen

    # Simpan hasil ke HDF5
    gen_dispatch.to_parquet(f'{file_dir}/scenario_{int(load_lvl_iter*100)}.parquet', index=False)
    print(f'{load_lvl_iter:.2f} load, OPF and LF running successfully')

0.50 load, OPF and LF running successfully
0.55 load, OPF and LF running successfully
0.60 load, OPF and LF running successfully
0.65 load, OPF and LF running successfully
0.70 load, OPF and LF running successfully
0.75 load, OPF min los failed, trying min dev
0.75 load, OPF and LF running successfully
0.80 load, OPF and LF running successfully
0.85 load, OPF and LF running successfully
0.90 load, OPF and LF running successfully
0.95 load, OPF and LF running successfully
1.00 load, OPF and LF running successfully
1.05 load, OPF and LF running successfully
1.10 load, OPF and LF running successfully


In [16]:
import os
import itertools
import pandas as pd
from tqdm import tqdm
import time

# Initialize variables
dataset_subdirs = ['magnitude', 'angle', 'time', 'out_of_step']
dataset_dir = 'output/dataset'

# Create directories if they don't exist
for dir in dataset_subdirs:
    os.makedirs(f'{dataset_dir}/{dir}', exist_ok=True)

# Prepare an empty list to store iteration data
iteration_data = []

# Pre-compute total iterations
total_iterations = len(load_level) * len(f_line) * len(f_location) * len(f_duration)
num_digits = len(str(total_iterations))

# Start progress bar
with tqdm(total=total_iterations, desc="Simulating", unit="iter") as pbar:
    iteration_counter = 1
    for load_lvl_iter in load_level:
        # Perform load setup and OPF analysis
        sim.loadSetup(load_lvl_iter, load_default)
        try:
            sim.opfAnalysis('los')
        except AttributeError:
            sim.opfAnalysis('dev')
        
        # Perform short circuit setup and RMS simulation
        for f_line_iter, f_loc_iter, f_dur_iter in itertools.product(f_line, f_location, f_duration):
            # Start iteration timing
            iter_start_time = pbar.format_dict['elapsed']
            filename = f'scenario_{iteration_counter:0{num_digits}d}'

            # Record the iteration time
            iter_end_time = pbar.format_dict['elapsed']
            iter_time = iter_end_time - iter_start_time

            # Save iteration data to the list
            iteration_data.append({
                "scenario": filename,
                "load_level": load_lvl_iter,
                "f_line": f_line_iter.loc_name,
                "f_location": f_loc_iter,
                "f_duration": f_dur_iter,
                "iter_time": iter_time
            })

            # Update tqdm progress bar
            pbar.set_postfix({
                "current_scenario": filename,
                "completed": f"{pbar.n}/{total_iterations}"
            })
            pbar.update(1)
            iteration_counter += 1
            
            time.sleep(0.05)

# After the loop finishes, create a DataFrame from the iteration data
df = pd.DataFrame(iteration_data)

# Optionally, save the DataFrame to a CSV file
df.to_csv('iteration_times.csv', index=False)

print("Simulation completed. Iteration times saved to 'iteration_times.csv'.")


Simulating: 100%|██████████| 8840/8840 [07:47<00:00, 18.91iter/s, current_scenario=scenario_8840, completed=8839/8840]

Simulation completed. Iteration times saved to 'iteration_times.csv'.


In [9]:
from tqdm import tqdm
import time

for i in tqdm(range(10)):
    time.sleep(0.5)


100%|██████████| 10/10 [00:05<00:00,  1.99it/s]


In [4]:
for load_lvl_iter in load_level:
    filename_load = 'Kondisi %d persen beban'%(int(round(load_lvl_iter*100,0)))
    if exists(join(parentAddress,'csv',filename_load+'.xlsx')):
        print(filename_load+' exist')
    else:
        print(filename_load+' does not exist')
        sim.loadSetup(load_lvl_iter, load_ori)
        try:
            sim.opfAnalysis()
            sim.ldfAnalysis()
        except AttributeError:
            sim.ldfAnalysis()
        sim.saveLoadandGen(load_lvl_iter, join(parentAddress,'csv'))
    for f_line_iter, f_loc_iter, f_dur_iter in itertools.product(f_line, f_location, f_duration):
        time_start=perf_counter()
        k += 1
        filename = '%s %.0f percent %.2f s %.2f load'%(f_line_iter.loc_name, f_loc_iter, f_dur_iter, load_lvl_iter)
        if exists(join(parentAddress,'csv','rotor_angle',filename+'.csv')):
            print(filename+' exist')
        else:
            print(filename+' does not exist')
            
            sim.shortCircuitSetup(f_loc_iter, f_line_iter, f_dur_iter)
            sim.simulationRMS(MONITORED_VARIABLES, t_start=-100, t_step=10, t_stop=30)
            
            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmSym',
                var_name = [MONITORED_VARIABLES['*.ElmSym'][0]],
                var_folder = [VAR_FOLDER['*.ElmSym'][0]],
                dir_address = join(parentAddress,"csv"))
            
            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmSym',
                var_name = [MONITORED_VARIABLES['*.ElmSym'][1]],
                var_folder = [VAR_FOLDER['*.ElmSym'][1]],
                dir_address = join(parentAddress,"csv"))
            
            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmRes',
                var_name = ['b:tnow'],
                var_folder = ['time'],
                dir_address = join(parentAddress,"csv"))
            
            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmTerm',
                var_name = [MONITORED_VARIABLES['*.ElmTerm'][0]],
                var_folder = [VAR_FOLDER['*.ElmTerm'][0]],
                dir_address = join(parentAddress,"csv"))

            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmTerm',
                var_name = [MONITORED_VARIABLES['*.ElmTerm'][1]],
                var_folder = [VAR_FOLDER['*.ElmTerm'][1]],
                dir_address = join(parentAddress,"csv"))
            
            print("%d / %d - %s %d percent %.2f s %.2f load"%(k,len(f_line)*len(f_location)*len(f_duration)*len(load_level),f_line_iter.loc_name,f_loc_iter,f_dur_iter,load_lvl_iter))
            time_end=perf_counter()
            minute,second=divmod((time_end-time_start),60)
            elapsed_hour, elapsed_sec = divmod((time_end-tstarttot), 3600)
            elapsed_min, elapsed_sec = divmod(elapsed_sec, 60)
            print("Progress %.2f persen - Waktu running per iterasi: %.2f detik - Waktu terlewati: %d jam %d menit %.2f detik"%(100*k/(len(f_line)*len(f_location)*len(f_duration)*len(load_level)),second,elapsed_hour,elapsed_min,elapsed_sec))
hour,second=divmod((time_end-tstarttot),3600)
minute,second=divmod(second,60)
print("Total waktu: %d jam %d menit %.2f detik"%(hour,minute,second))
sim.loadSetup(1, load_ori)

,name,parallel,V,pf,P,Q
0,G 01,1,1.015863,0.988703,602.284677,-91.308164
1,G 02,1,0.982000,0.966402,512.292188,136.254946
2,G 03,1,0.990396,0.998661,215.743747,11.177586
3,G 04,1,1.024547,0.999929,226.007788,2.685207
4,G 05,2,1.025169,0.990644,226.332616,31.179573
5,G 06,1,1.020712,0.999990,385.692937,-1.738357
6,G 07,1,1.022495,0.999879,168.582075,2.621380
7,G 08,1,1.012804,0.973286,205.317333,-48.433893
8,G 09,1,1.038461,0.934800,291.751331,-110.850112
9,G 10,1,1.007967,0.930295,221.584282,-87.370294
